In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate
    from ms.market_size_daily_estimate_1000 
    where 
        date = '2020-01-10' and 
        purchase_type_id=12 and 
        app_type_id=2 and 
        store_id=14 and 
        category_id in (65) 
\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
EOF


In [0]:
%%sh

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 

SELECT date,count(*),count(distinct(app_price_type_id, purchase_type_id,category_id,device_code,country_code,date,granularity)) FROM store.store_market_size_fact_v1  WHERE date between '2020-09-21' AND '2020-09-22' GROUP BY date;

EOF

In [0]:
%%sh

PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 

SELECT date,count(distinct(app_id)) FROM store.store_est_fact_v6  WHERE date between '2014-01-01' AND '2014-05-30' AND device_code='ios-tablet' GROUP BY date;

EOF

In [0]:


import psycopg2
import datetime

today = datetime.datetime.today()
date_delta = z.input("max_days")
date_start = today + datetime.timedelta(0-int(date_delta)-1)


citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_store_db",
        user="citus_bdp_prod_app_int_qa",
        host="10.2.6.141",
        password="wZw8cfBuuklIskVG",
        port=5432
    )
)



sql = "SELECT date,device_code,count(distinct(app_id)) FROM store.store_est_fact_v6  WHERE date between '2014-01-01' AND '2014-01-02' GROUP BY date,device_code;"

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
test_result = query(citus_dsn_, sql)
print "%table {}\t{}\t{}".format( "date", "device_code", "count")

for result in test_result:
    date,device_code,count = result
    print "{}\t{}\t{}".format(date.strftime("%Y-%m-%d"), device_code, count)



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136 -U app_bdp_usage_qa -d dna -p 6432 << EOF 

select * from in_sdk_product_mapping limit 5;

\d

EOF